## Example Usage the HecTime Class ##

### Setup ###

In [8]:
import os, sys

srcdir = os.path.join(os.path.dirname(os.getcwd()), "src")
sys.path.append(srcdir)
from hec.hectime import HecTime  # type: ignore
from hec.interval import Interval  # type: ignore
from IPython.display import display_html  # type: ignore

### End-of-month 1Month intervals for a time window ###

In [9]:
intvl = Interval.getDss("1Month")
start_time = HecTime("2024-01-31, 24:00")
end_time = start_time + "1Y"
count = start_time.computeNumberIntervals(end_time, intvl)
times = [start_time + f"{i}M" for i in range(count)]
for t in times:
    print(t)

2024-01-31T24:00:00
2024-02-29T24:00:00
2024-03-31T24:00:00
2024-04-30T24:00:00
2024-05-31T24:00:00
2024-06-30T24:00:00
2024-07-31T24:00:00
2024-08-31T24:00:00
2024-09-30T24:00:00
2024-10-31T24:00:00
2024-11-30T24:00:00
2024-12-31T24:00:00


### LRTS Daily Times in UTC ###
Note the transition from 13:00 to 12:00 (23 hour day) in the Spring DST transition and then back again (25 hour day) in the Autum DST transistion.

In [10]:
intvl = Interval.getCwms("1Day")
offset = 7 * Interval.MINUTES["1Hour"]
for month in ("Mar", "Nov"):
    start_time = HecTime(f"01{month}2024, 00:00").atTimeZone("US/Central")
    end_time = start_time + "1M"
    first_time = HecTime(start_time)
    if first_time.adjustToIntervalOffset(intvl, offset) < start_time:
        first_time.increment(1, intvl)
    count = first_time.computeNumberIntervals(end_time, intvl)
    times = [(first_time + f"{i}D").astimezone("UTC") for i in range(count)]
    prev = None
    for t in times:
        if prev:
            print(f"{t}\t{(t-prev).total_seconds() // 3600}")
        else:
            print(t)
        prev = t
    print("")

2024-03-01T13:00:00+00:00
2024-03-02T13:00:00+00:00	24
2024-03-03T13:00:00+00:00	24
2024-03-04T13:00:00+00:00	24
2024-03-05T13:00:00+00:00	24
2024-03-06T13:00:00+00:00	24
2024-03-07T13:00:00+00:00	24
2024-03-08T13:00:00+00:00	24
2024-03-09T13:00:00+00:00	24
2024-03-10T12:00:00+00:00	23
2024-03-11T12:00:00+00:00	24
2024-03-12T12:00:00+00:00	24
2024-03-13T12:00:00+00:00	24
2024-03-14T12:00:00+00:00	24
2024-03-15T12:00:00+00:00	24
2024-03-16T12:00:00+00:00	24
2024-03-17T12:00:00+00:00	24
2024-03-18T12:00:00+00:00	24
2024-03-19T12:00:00+00:00	24
2024-03-20T12:00:00+00:00	24
2024-03-21T12:00:00+00:00	24
2024-03-22T12:00:00+00:00	24
2024-03-23T12:00:00+00:00	24
2024-03-24T12:00:00+00:00	24
2024-03-25T12:00:00+00:00	24
2024-03-26T12:00:00+00:00	24
2024-03-27T12:00:00+00:00	24
2024-03-28T12:00:00+00:00	24
2024-03-29T12:00:00+00:00	24
2024-03-30T12:00:00+00:00	24

2024-11-01T12:00:00+00:00
2024-11-02T12:00:00+00:00	24
2024-11-03T13:00:00+00:00	25
2024-11-04T13:00:00+00:00	24
2024-11-05T13:00:00

### Show Interval Boundaries for Current Time for All Intervals ###

In [11]:
t = HecTime.now()
t.midnight_as_2400 = False
print("Current Time\t\tIntvl\tTop of Prev\t\tTop of Next")
for intvl in Interval.getAll(lambda i: i.minutes > 0):
    prev = HecTime(t).adjustToIntervalOffset(intvl, 0)
    next = HecTime(prev).increment(1, intvl)
    print(f"{t}\t{intvl}\t{prev}\t{next}")

Current Time		Intvl	Top of Prev		Top of Next
2024-10-01T14:27:00	PT1M	2024-10-01T14:27:00	2024-10-01T14:28:00
2024-10-01T14:27:00	PT2M	2024-10-01T14:26:00	2024-10-01T14:28:00
2024-10-01T14:27:00	PT3M	2024-10-01T14:27:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT4M	2024-10-01T14:24:00	2024-10-01T14:28:00
2024-10-01T14:27:00	PT5M	2024-10-01T14:25:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT6M	2024-10-01T14:24:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT10M	2024-10-01T14:20:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT12M	2024-10-01T14:24:00	2024-10-01T14:36:00
2024-10-01T14:27:00	PT15M	2024-10-01T14:15:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT20M	2024-10-01T14:20:00	2024-10-01T14:40:00
2024-10-01T14:27:00	PT30M	2024-10-01T14:00:00	2024-10-01T14:30:00
2024-10-01T14:27:00	PT1H	2024-10-01T14:00:00	2024-10-01T15:00:00
2024-10-01T14:27:00	PT2H	2024-10-01T14:00:00	2024-10-01T16:00:00
2024-10-01T14:27:00	PT3H	2024-10-01T12:00:00	2024-10-01T15:00:00
2024-10-01T14:27:00	PT4H	2024-10-01T12:0

### Format Date With Style (Generate Date Style Table) ###

In [12]:
width = 18
t = HecTime("02Jun1985, 12:34").atTimeZone("local")
html = '<html><pre><table border="1"><tr><th colspan="4" style="text-align:left;"><b>Base Styles</b></th></tr>'
for row in range(10):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row + col
        html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += '<tr><th colspan="4" style="text-align:left;"><b>Extended Styles</b></th></tr>'
for row in (-1, -2, -3):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row - col
        if style in (-3, -103, -113):
            html += "<td></td>"
        else:
            html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += "</table></pre></html>"
display_html(html, raw=True)  # type: ignore

Base Styles 0: June 2, 1985 10: June 2, 85 100: JUNE 2, 1985 110: JUNE 2, 85 1: Jun 2, 1985 11: Jun 2, 85 101: JUN 2, 1985 111: JUN 2, 85 2: 2 June 1985 12: 2 June 85 102: 2 JUNE 1985 112: 2 JUNE 85 3: June 1985 13: June 85 103: JUNE 1985 113: JUNE 85 4: 02Jun1985 14: 02Jun85 104: 02JUN1985 114: 02JUN85 5: 2Jun1985 15: 2Jun85 105: 2JUN1985 115: 2JUN85 6: Jun1985 16: Jun85 106: JUN1985 116: JUN85 7: 02 Jun 1985 17: 02 Jun 85 107: 02 JUN 1985 117: 02 JUN 85 8: 2 Jun 1985 18: 2 Jun 85 108: 2 JUN 1985 118: 2 JUN 85 9: Jun 1985 19: Jun 85 109: JUN 1985 119: JUN 85 Extended Styles -1: 6/2/85 -11: 06/02/85 -101: 6/2/1985 -111: 06/02/1985 -2: 6-2-85 -12: 06-02-85 -102: 6-2-1985 -112: 06-02-1985 -13: 1985-06-02

### Format and Parse Date/Time Like `datetime.datetime` ###

In [13]:
t = HecTime("02Jun1985, 12:34").atTimeZone("local")
print(f"HecTime object = {t}")
format = "%A, %B %d, %Y %H:%M:%S %z"
dateTimeStr = t.strftime(format)
print(f'Formatted using strftime("{format}")  : {dateTimeStr}')
t.setUndefined()
t.atTimeZone(None)
print(f"\nNew HecTime Object : {repr(t)}")
print(
    f'After parsing "{dateTimeStr}" with "{format}"  : {t.strptime(dateTimeStr, format)}'
)

HecTime object = 1985-06-02T12:34:00-05:00
Formatted using strftime("%A, %B %d, %Y %H:%M:%S %z")  : Sunday, June 02, 1985 12:34:00 -0500

New HecTime Object : HecTime(UNDEFINED_TIME, MINUTE_GRANULARITY)
After parsing "Sunday, June 02, 1985 12:34:00 -0500" with "%A, %B %d, %Y %H:%M:%S %z"  : 1985-06-02T12:34:00-05:00
